In [1]:
import json
import pandas as pd

In [2]:
import os
from gensim import matutils
from pyvi import ViTokenizer
import string
import re
import numpy as np
import regex as re
import math

class Char:
    UNICHARS = "àáảãạâầấẩẫậăằắẳẵặèéẻẽẹêềếểễệđìíỉĩịòóỏõọôồốổỗộơờớởỡợùúủũụưừứửữựỳýỷỹỵÀÁẢÃẠÂẦẤẨẪẬĂẰẮẲẴẶÈÉẺẼẸÊỀẾỂỄỆĐÌÍỈĨỊÒÓỎÕỌÔỒỐỔỖỘƠỜỚỞỠỢÙÚỦŨỤƯỪỨỬỮỰỲÝỶỸỴÂĂĐÔƠƯ"
    UNSIGNCHARS = "aaaaaaaaaaaaaaaaaeeeeeeeeeeediiiiiooooooooooooooooouuuuuuuuuuuyyyyyAAAAAAAAAAAAAAAAAEEEEEEEEEEEDIIIOOOOOOOOOOOOOOOOOOOUUUUUUUUUUUYYYYYAADOOU"
    CHAR1252 = 'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    CHARUTF8 = "à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ"
    VOWEL_TABLE = [['a', 'à', 'á', 'ả', 'ã', 'ạ', 'a'],
                   ['ă', 'ằ', 'ắ', 'ẳ', 'ẵ', 'ặ', 'aw'],
                   ['â', 'ầ', 'ấ', 'ẩ', 'ẫ', 'ậ', 'aa'],
                   ['e', 'è', 'é', 'ẻ', 'ẽ', 'ẹ', 'e'],
                   ['ê', 'ề', 'ế', 'ể', 'ễ', 'ệ', 'ee'],
                   ['i', 'ì', 'í', 'ỉ', 'ĩ', 'ị', 'i'],
                   ['o', 'ò', 'ó', 'ỏ', 'õ', 'ọ', 'o'],
                   ['ô', 'ồ', 'ố', 'ổ', 'ỗ', 'ộ', 'oo'],
                   ['ơ', 'ờ', 'ớ', 'ở', 'ỡ', 'ợ', 'ow'],
                   ['u', 'ù', 'ú', 'ủ', 'ũ', 'ụ', 'u'],
                   ['ư', 'ừ', 'ứ', 'ử', 'ữ', 'ự', 'uw'],
                   ['y', 'ỳ', 'ý', 'ỷ', 'ỹ', 'ỵ', 'y']]

    FIRST_CHAR_TABLE = ['', 'f', 's', 'r', 'x', 'j']
    REGEX_UNICODE_TEMPLATE = r'à|á|ả|ã|ạ|ầ|ấ|ẩ|ẫ|ậ|ằ|ắ|ẳ|ẵ|ặ|è|é|ẻ|ẽ|ẹ|ề|ế|ể|ễ|ệ|ì|í|ỉ|ĩ|ị|ò|ó|ỏ|õ|ọ|ồ|ố|ổ|ỗ|ộ|ờ|ớ|ở|ỡ|ợ|ù|ú|ủ|ũ|ụ|ừ|ứ|ử|ữ|ự|ỳ|ý|ỷ|ỹ|ỵ|À|Á|Ả|Ã|Ạ|Ầ|Ấ|Ẩ|Ẫ|Ậ|Ằ|Ắ|Ẳ|Ẵ|Ặ|È|É|Ẻ|Ẽ|Ẹ|Ề|Ế|Ể|Ễ|Ệ|Ì|Í|Ỉ|Ĩ|Ị|Ò|Ó|Ỏ|Õ|Ọ|Ồ|Ố|Ổ|Ỗ|Ộ|Ờ|Ớ|Ở|Ỡ|Ợ|Ù|Ú|Ủ|Ũ|Ụ|Ừ|Ứ|Ử|Ữ|Ự|Ỳ|Ý|Ỷ|Ỹ|Ỵ'
    EMOJI_PATTERN = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               "]+", flags=re.UNICODE)

uniChars = Char.UNICHARS
unsignChars = Char.UNSIGNCHARS
PUNCT_TO_REMOVE = string.punctuation

vowel_table = Char.VOWEL_TABLE
first_char_table = Char.FIRST_CHAR_TABLE
vowel_to_ids = {}


def loaddicchar():
    dic = {}
    char1252 = Char.CHAR1252.split(
        '|')
    charutf8 = Char.CHARUTF8.split(
        '|')
    for i in range(len(char1252)):
        dic[char1252[i]] = charutf8[i]
    return dic


dicchar = loaddicchar()


def gen_text(object):
    res = []
    if object['detailed_address'] == object['detailed_address']:
        text = f"Bán nhà tại {object['detailed_address']}"
        res.append(text)
    if math.isnan(object['number_of_floors']):
        res.append(f"Xây {object['number_of_floors']} tầng")
        res.append(f"{object['number_of_floors']}t")
        res.append(f"Thiết kế xây {object['number_of_floors']}t")
        res.append(f"Thiết kế xây {object['number_of_floors']} tầng")

    if math.isnan(object['acreage']):
        res.append(f"Diện tích {object['acreage']}m²")
        res.append(f"{object['acreage']}m²")
        if object['detailed_address'] == object['detailed_address']:
            res.append(
                f"Bán nhà tại {object['detailed_address']}, {object['acreage']}m")

    if math.isnan(object['facede']):
        res.append(f"{object['facede']}M MT")

    if math.isnan(object['number_of_bedrooms']):
        res.append(f"Nhà có {object['facede']} phòng ngủ")
        res.append(f"{object['facede']} phòng ngủ")

    if math.isnan(object['is_car_road']):
        if object['is_car_road'] == 0:
            res.append(
                f"Nhà cách ngõ ô tô tránh {object['cach_ngo_o_to_tranh']}m")
            res.append(
                f"Cách {object['cach_ngo_o_to_tranh']}m ra ngõ ô tô tránh")
            res.append(
                f"Cách Mặt Ngõ Ô Tô Tránh {object['cach_ngo_o_to_tranh']}m")
        elif object['is_car_road'] == 1:
            res.append(f"Nhà ở ngay mặt ngõ")
            res.append(f"Nhà ở mặt ngõ")

    res = '. '.join(res)
    res = res.strip()

    return res


for i in range(len(vowel_table)):
    for j in range(len(vowel_table[i]) - 1):
        vowel_to_ids[vowel_table[i][j]] = (i, j)


def standardize_vietnamese_word_sign(word):
    if not is_valid_vietnam_word(word):
        return word

    chars = list(word)
    sentence_sign = 0
    vowel_index = []
    qu_or_gi = False
    for index, char in enumerate(chars):
        x, y = vowel_to_ids.get(char, (-1, -1))
        if x == -1:
            continue
        elif x == 9:  # check qu
            if index != 0 and chars[index - 1] == 'q':
                chars[index] = 'u'
                qu_or_gi = True
        elif x == 5:  # check gi
            if index != 0 and chars[index - 1] == 'g':
                chars[index] = 'i'
                qu_or_gi = True
        if y != 0:
            sentence_sign = y
            chars[index] = vowel_table[x][0]
        if not qu_or_gi or index != 1:
            vowel_index.append(index)
    if len(vowel_index) < 2:
        if qu_or_gi:
            if len(chars) == 2:
                x, y = vowel_to_ids.get(chars[1])
                chars[1] = vowel_table[x][sentence_sign]
            else:
                x, y = vowel_to_ids.get(chars[2], (-1, -1))
                if x != -1:
                    chars[2] = vowel_table[x][sentence_sign]
                else:
                    chars[1] = vowel_table[5][sentence_sign] if chars[1] == 'i' else vowel_table[9][sentence_sign]
            return ''.join(chars)
        return word

    for index in vowel_index:
        x, y = vowel_to_ids[chars[index]]
        if x == 4 or x == 8:  # ê, ơ
            chars[index] = vowel_table[x][sentence_sign]
            # for index2 in vowel_index:
            #     if index2 != index:
            #         x, y = vowel_to_ids[chars[index]]
            #         chars[index2] = vowel_table[x][0]
            return ''.join(chars)

    if len(vowel_index) == 2:
        if vowel_index[-1] == len(chars) - 1:
            x, y = vowel_to_ids[chars[vowel_index[0]]]
            chars[vowel_index[0]] = vowel_table[x][sentence_sign]
            # x, y = vowel_to_ids[chars[vowel_index[1]]]
            # chars[vowel_index[1]] = vowel_table[x][0]
        else:
            # x, y = vowel_to_ids[chars[vowel_index[0]]]
            # chars[vowel_index[0]] = vowel_table[x][0]
            x, y = vowel_to_ids[chars[vowel_index[1]]]
            chars[vowel_index[1]] = vowel_table[x][sentence_sign]
    else:
        # x, y = vowel_to_ids[chars[vowel_index[0]]]
        # chars[vowel_index[0]] = vowel_table[x][0]
        x, y = vowel_to_ids[chars[vowel_index[1]]]
        chars[vowel_index[1]] = vowel_table[x][sentence_sign]
        # x, y = vowel_to_ids[chars[vowel_index[2]]]
        # chars[vowel_index[2]] = vowel_table[x][0]
    return ''.join(chars)


def is_valid_vietnam_word(word):
    chars = list(word)
    vowel_index = -1
    for index, char in enumerate(chars):
        x, y = vowel_to_ids.get(char, (-1, -1))
        if x != -1:
            if vowel_index == -1:
                vowel_index = index
            else:
                if index - vowel_index != 1:
                    return False
                vowel_index = index
    return True


def standardize_vietnamese_sentence_sign(sentence):

    sentence = sentence.lower()
    words = sentence.split()
    for index, word in enumerate(words):
        cw = re.sub(r'(^\p{P}*)([p{L}.]*\p{L}+)(\p{P}*$)',
                    r'\1/\2/\3', word).split('/')
        # print(cw)
        if len(cw) == 3:
            cw[1] = standardize_vietnamese_word_sign(cw[1])
        words[index] = ''.join(cw)
    return ' '.join(words)


def covert_unicode(txt):
    return re.sub(
        Char.REGEX_UNICODE_TEMPLATE,
        lambda x: dicchar[x.group()], txt)


def preprocess_text(text):
    try:
        text = text.lower()
        text = covert_unicode(text)
        text = standardize_vietnamese_sentence_sign(text)
        return text
    except:
        return np.nan


def remove_emoji(text):
    emoji_pattern = Char.EMOJI_PATTERN
    return emoji_pattern.sub(r'', text)


def remove_special_character(text):
    text = text.lower()
    text = text.translate(str.maketrans('', '', PUNCT_TO_REMOVE))
    text = text.replace('\n', '')
    text = text.strip()
    words = text.split(" ")
    words = [word for word in words if word != ""]
    return " ".join(words)




In [3]:
data = json.load(open('../data/process_v1/process_data.json', 'r'))
data[0]

{'numberOfFloors': 1,
 'numberOfBathRooms': 0,
 'numberOfBedRooms': 0,
 'numberOfKitchens': 0,
 'numberOfGarages': 0,
 'certificateOfLandUseRight': 'yes',
 'ward': 'Văn Quán',
 'street': 'Chiến Thắng',
 'district': 'Hà Đông',
 'city': 'Hà Nội',
 'lat': 20.978546,
 'lon': 105.796775,
 'typeOfRealEstate': 'privateProperty',
 'frontWidth': 0,
 'endWidth': 0,
 'facade': 'oneSideOpen',
 'houseDirection': None,
 'landSize': 48,
 'price': 4.35,
 'unitPrice': 'billion',
 'distanceToNearestRoad': 0,
 'frontRoadWidth': 0,
 'accessibility': 'fitTwoCars',
 'landType': 'residentialLand',
 'description': '<div>-\tBán gấp nhà đường Chiến Thắng, đầu quận Hà Đông, dễ dàng vào trung tâm HN, sang quận Thanh Xuân, Nam Từ Liêm, gần các trường ĐH An Ninh, Bưu chính…</div><div>- Diện tích 48m2, phân lô vuông vức, ô tô đỗ tận cửa, khu vực dân trí cao, văn minh, nhà có thể ở ngay.</div><div>-\tSổ đỏ sạch đẹp sẵn sàng giao dịch.</div><div>-\tCam kết: không thu bất kể loại phí gì kể cả khi giao dịch thành công.<

In [4]:
data = [item for item in data if item['street'] != None]

In [5]:
len(data)

236192

In [6]:
data = [item for item in data if item['district'] != None]

In [7]:
len(data)

236192

In [8]:
data = [item for item in data if item['lat'] != None and item['lon'] != None]

In [9]:
data = [item for item in data if item['landSize'] != None and item['price'] != None]

In [10]:
len(data)

236192

In [11]:
data = pd.DataFrame(data)

In [12]:
data

,numberOfFloors,numberOfBathRooms,numberOfBedRooms,numberOfKitchens,numberOfGarages,certificateOfLandUseRight,ward,street,district,city,...,houseDirection,landSize,price,unitPrice,distanceToNearestRoad,frontRoadWidth,accessibility,landType,description,time
0,1,0,0,0,0,yes,Văn Quán,Chiến Thắng,Hà Đông,Hà Nội,...,None,48.0,4.35,billion,0,0,fitTwoCars,residentialLand,"<div>-\tBán gấp nhà đường Chiến Thắng, đầu quậ...",2021-07-20T02:57:00
1,1,0,0,0,0,yes,Văn Quán,Chiến Thắng,Hà Đông,Hà Nội,...,None,40.0,4.68,billion,0,0,fitOneCarAndOneMotorbike,residentialLand,Đang làm ăn lớn bán đất vị trí thuận lợi tọa l...,2023-02-28T22:48:42
2,5,0,4,0,0,yes,Văn Quán,Trần Phú,Hà Đông,Hà Nội,...,None,40.0,5.59,billion,0,0,notInTheAlley,residentialLand,"Bán nhà vào ở ngay giá mềm chỉ 5.59 tỷ, có diệ...",2021-12-21T17:00:01
3,5,0,3,0,0,yes,Phúc La,19/5,Hà Đông,Hà Nội,...,None,35.0,5.15,billion,0,0,notInTheAlley,residentialLand,"Thất nghiệp bán biệt thự, giá bán cực mềm 5.15...",2021-10-31T22:50:48
4,5,0,8,0,0,yes,Văn Quán,Đại An,Hà Đông,Hà Nội,...,None,55.0,4.70,billion,0,0,fitOneCarAndOneMotorbike,residentialLand,"Bán nhà giá siêu khủng chỉ 4.7 tỷ,diện tích kh...",2020-12-02T02:47:43
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
236187,1,2,3,0,0,yes,,,Nam Từ Liêm,Hà Nội,...,None,105.0,0.00,billion,0,0,notInTheAlley,residentialLand,Chủ nhà thiện chí cần bán gấp căn 105m² tại tò...,2024-02-16T00:00:00
236188,1,0,0,0,0,yes,,,Long Biên,Hà Nội,...,None,75.0,19.00,billion,0,0,fitThreeCars,residentialLand,"Bán đất hiếm! Đất mặt phố Cổ Linh, vị trí gần ...",2024-02-16T00:00:00
236189,1,0,0,0,0,yes,,,Hà Đông,Hà Nội,...,None,32.0,4.95,billion,0,0,notInTheAlley,residentialLand,"Vị trí đất siêu đắc địa, ngõ thông rộng 6m kin...",2024-02-16T00:00:00
236190,1,2,2,0,0,no,,,Long Biên,Hà Nội,...,None,70.0,2.00,billion,0,0,notInTheAlley,residentialLand,Chính chủ gửi bán gấp căn hộ 70m² 2 ngủ 2 vs n...,2024-02-15T00:00:00


In [13]:
data['district'] = data['district'].apply(preprocess_text)

In [14]:
data['city'] = data['city'].apply(preprocess_text)

In [15]:
data = data[data['city'].isin(['hà nội', 'hồ chí minh'])]

In [16]:
len(data)

209713

In [28]:
item = list(data[data['city'] == 'hồ chí minh']['ward'].unique())
import json

dictionary = [i for i in item if i != '']

with open("./test.json", "w", encoding='utf8') as outfile:
    json.dump(dictionary, outfile, ensure_ascii=False)

In [ ]:
hn_df = data[data['city'] == 'hà nội']
hn_df['district'] = hn_df['district'].replace('mê linh', 'suburb_west')
hn_df['district'] = hn_df['district'].replace('ba vì', 'suburb_west')
hn_df['district'] = hn_df['district'].replace('phúc thọ', 'suburb_west')
hn_df['district'] = hn_df['district'].replace('thạch thất', 'suburb_west')
hn_df['district'] = hn_df['district'].replace('mỹ đức', 'suburb_west')
hn_df['district'] = hn_df['district'].replace('sơn tây', 'suburb_west')
hn_df['district'] = hn_df['district'].replace('quốc oai', 'suburb_west')
hn_df['district'] = hn_df['district'].replace('quốc oai', 'suburb_west')

hn_df['district'] = hn_df['district'].replace('sóc sơn', 'suburb_north')
hn_df['district'] = hn_df['district'].replace('đan phượng', 'suburb_north')

hn_df['district'] = hn_df['district'].replace('thanh oai', 'suburb_south')
hn_df['district'] = hn_df['district'].replace('ứng hòa', 'suburb_south')
hn_df['district'] = hn_df['district'].replace('phú xuyên', 'suburb_south')
hn_df['district'] = hn_df['district'].replace('thường tín', 'suburb_south')
hn_df['district'] = hn_df['district'].replace('chương mỹ', 'suburb_south')

In [ ]:
hn_df['district'].value_counts()

In [ ]:
hcm_df = data[data['city'] == 'hồ chí minh']

In [ ]:
hcm_df

In [ ]:
df = pd.concat([hn_df, hcm_df])

In [ ]:
df = df.sample(frac = 1.0)

In [ ]:
df

In [ ]:
df['ward'] = df['ward'].apply(preprocess_text)
df['street'] = df['street'].apply(preprocess_text)
df['description'] = df['description'].apply(preprocess_text)

In [ ]:
df

In [ ]:
df

In [ ]:
# df = df.reset_index(drop = True)
# df.to_csv('../data/process_v1/process_data_2.csv', index = False)